DataScience task 3 (Next word prediction)

In [3]:
pip install tensorflow


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

# Sample text data
text = """
You may say I'm a dreamer, but I'm not the only one. I hope someday you'll join us. And the world will live as one.
"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and create predictors and labels
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [9]:
# Build the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# View model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 24, 100)           2300      
                                                                 
 bidirectional_1 (Bidirecti  (None, 24, 300)           301200    
 onal)                                                           
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 23)                2323      
                                                                 
Total params: 466223 (1.78 MB)
Trainable params: 466223 (1.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Train the model
history = model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 3.1364 - accuracy: 0.0833
Epoch 2/100
1/1 [==============================] - 0s 211ms/step - loss: 3.1248 - accuracy: 0.0833
Epoch 3/100
1/1 [==============================] - 0s 197ms/step - loss: 3.1125 - accuracy: 0.1667
Epoch 4/100
1/1 [==============================] - 0s 186ms/step - loss: 3.0972 - accuracy: 0.1667
Epoch 5/100
1/1 [==============================] - 0s 218ms/step - loss: 3.0765 - accuracy: 0.1250
Epoch 6/100
1/1 [==============================] - 0s 218ms/step - loss: 3.0468 - accuracy: 0.1250
Epoch 7/100
1/1 [==============================] - 0s 209ms/step - loss: 3.0045 - accuracy: 0.1250
Epoch 8/100
1/1 [==============================] - 0s 200ms/step - loss: 2.9528 - accuracy: 0.1250
Epoch 9/100
1/1 [==============================] - 0s 186ms/step - loss: 2.9245 - accuracy: 0.1250
Epoch 10/100
1/1 [==============================] - 0s 125ms/step - loss: 2.8943 - accuracy: 0.1250
Epoch 11/100

In [11]:
import numpy as np

def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    return predicted_word

# Example usage
seed_text = "You may say I'm"
next_words = 10

for _ in range(next_words):
    next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
    seed_text += " " + next_word

print(seed_text)


You may say I'm a dreamer but i'm not the only one i hope
